In [1]:
import numpy as np
import pandas as pd
from keras.models import load_model
import pickle

In [3]:
#Read patient info, given by vera
f = open('testPatient.pkl','r')
testPatient = pickle.load(f)
f.close()

In [5]:
#Read model, saved after training, Change to file containing the final model
model = load_model('Clindata_LSTM_Test.h5')

In [7]:
#Correct shape of test Patient
def correctInputShape(data_x):
    size = data_x.shape[0]
    num_features = data_x[0].shape[1]
    num_drugs = data_x[0].shape[0]
    
    x_shaped = np.empty((size, num_drugs, num_features))
    
    for i in range(size):
        x_shaped[i] = np.nan_to_num(data_x[i].todense())
    
    return x_shaped

testPatient = correctInputShape(testPatient)

In [35]:
#Predict probabilities
predicted_proba = model.predict(testPatient)

#Extract probabilities from the last drug
last_pred = predicted_proba[len(predicted_proba)-1][0]

In [42]:
#Extract top 10 indices
inds = np.argsort(-last_pred)[:10]

In [36]:
#Read reaction names (this file is on the Github/Data Cleaning folder)
reactions = pd.read_csv('Top2000Reactions.csv')

In [43]:
#Extract top 10 reactions
reacts = [reactions['Reaction'][i] for i in inds]

#Extract top 10 probabilities
probs = [last_pred[i] for i in inds]

In [44]:
#put everything together in a dataframe
prediction_df = pd.DataFrame()
prediction_df['Reaction'] = reacts
prediction_df['Probability'] = probs
prediction_df

,Reaction,Probability
0,GASTRIC DISORDER,0.602004
1,DYSGEUSIA,0.595148
2,MULTIPLE SCLEROSIS RELAPSE,0.568862
3,HYPOGAMMAGLOBULINAEMIA,0.557208
4,CANDIDA INFECTION,0.556080
5,CATAPLEXY,0.552502
6,JUGULAR VEIN THROMBOSIS,0.546205
7,VISUAL FIELD DEFECT,0.546016
8,BALANCE DISORDER,0.541265
9,WEIGHT FLUCTUATION,0.541206
